# SDC1 Solution example workflow

This notebook contains an example workflow for solving SDC1. Running this workflow on the full set of SDC1 data is resource intensive, requiring ~100GB RAM, though the same process can be run on smaller sample data.

Some sample data can be downloaded into this notebook environment via running the script `bash binder/download_sample_data.sh` in a terminal. By default this places data in `data/sample_images`, which is the directory assumed by this workflow.

In [1]:
import os
from pathlib import Path

import numpy as np
from ska_sdc import Sdc1Scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from ska.sdc1.models.sdc1_image import Sdc1Image
from ska.sdc1.utils.bdsf_utils import cat_df_from_srl_df, load_truth_df
from ska.sdc1.utils.classification import SKLClassification
from ska.sdc1.utils.source_finder import SourceFinder
import pandas


---

In [2]:

# Challenge frequency bands
#
FREQS = [1400]


# Input data paths; assumes defaults from download_data.sh
#
print("data", "sample_images", "{}mhz_1000h.fits".format(500))
def image_path(freq):
    return os.path.join("data", "sample_images", "{}mhz_1000h.fits".format(freq))


def pb_path(freq):
    return os.path.join("data", "sample_images", "{}mhz_pb.fits".format(freq))


def train_truth_path(freq):
    return os.path.join("/idia/users/hussein/data", "truth", "{}mhz_truth_train.txt".format(freq))


def full_truth_path(freq):
    return os.path.join("/idia/users/hussein/data", "truth", "{}mhz_truth_full.txt".format(freq))


# Output data paths
#
def train_source_df_path(freq):
    return os.path.join("data", "sources", "{}mhz_sources_train.csv".format(freq))


def full_source_df_path(freq):
    return os.path.join("data", "sources", "{}mhz_sources_full.csv".format(freq))


def submission_df_path(freq):
    return os.path.join("data", "sources", "{}mhz_submission.csv".format(freq))


def model_path(freq):
    return os.path.join("data", "models", "{}mhz_classifier.pickle".format(freq))


def score_report_path(freq):
    return os.path.join("data", "score", "{}mhz_score.txt".format(freq))


def write_df_to_disk(df, out_path):
    """ Helper function to write DataFrame df to a file at out_path"""
    out_dir = os.path.dirname(out_path)
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    df.to_csv(out_path, index=False)



data sample_images 500mhz_1000h.fits


In [5]:
"""
Run through a simple analysis workflow to solve SDC1

1) Preprocess images (correct PB) and crop out the training area for
    building ML model
2) Find sources in the PB-corrected training images
3) Train a classifier for each band to predict the class of each source
4) Find sources in the full PB-corrected image
5) Predict the class of each source
6) Calculate the score for each image band, and write out a short report
"""
# 1) Create in-memory representation of image and preprocess
print("\nStep 1: Preprocessing")
sdc1_image_list = []
for freq in FREQS:
    new_image = Sdc1Image(freq, image_path(freq), pb_path(freq))
    new_image.preprocess()
    sdc1_image_list.append(new_image)

# In data/images, we now have PB-corrected and training images for each band




Step 1: Preprocessing


 [astropy.io.fits.verify]


In [6]:
sdc1_image_list

In [7]:

# 2) Source finding (training):
print("\nStep 2: Source finding (train)")
sources_training = {}
for sdc1_image in sdc1_image_list:
    source_finder = SourceFinder(sdc1_image.train)
    print(source_finder.image_dirname)
    sl_df = source_finder.run()
    sources_training[sdc1_image.freq] = sl_df

    # (Optional) Write source list DataFrame to disk
    write_df_to_disk(sl_df, train_source_df_path(sdc1_image.freq))

    # Remove temp files:
    source_finder.reset()

# <Additional feature engineering of the source DataFrames can be performed here>



--> Opened '1400mhz_1000h_train.fits'
Image size .............................. : (3469, 3184) pixels
Number of channels ...................... : 1
Number of Stokes parameters ............. : 1
Beam shape (major, minor, pos angle) .... : (1.66667e-04, 1.66667e-04, 0.0) degrees



Step 2: Source finding (train)
data/sample_images


Frequency of image ...................... : 1400.000 MHz
Number of blank pixels .................. : 0 (0.0%)
Flux from sum of (non-blank) pixels ..... : 0.034 Jy
--> Calculating background rms and mean images
/idia/projects/hack4dev/dc1/vDC1/lib/python3.8/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)
Using user-specified rms_box ............ : (74, 19) pixels
--> Using 2D map for background rms
--> Variation in mean image significant
--> Using 2D map for background mean
Min/max values of background rms map .... : (6.40e-08, 1.38e-05) Jy/beam
Min/max values of background mean map ... : (-5.79e-08, 9.85e-06) Jy/beam
Minimum number of pixels per island ..... : 6
Number of i

Fitting islands with Gaussians .......... : [|] 0/6636

stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device


...

[-8G/Fitting islands with Gaussians .......... : [/] 6469/6636[-8G-Fitting islands with Gaussians .......... : [-] 6470/6636[-8G\Fitting islands with Gaussians .......... : [\] 6471/6636[-8G|Fitting islands with Gaussians .......... : [|] 6472/6636[-8G/Fitting islands with Gaussians .......... : [/] 6473/6636[-8G-

stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device


Fitting islands with Gaussians .......... : [-] 6474/6636[-8G\Fitting islands with Gaussians .......... : [\] 6475/6636[-8G|Fitting islands with Gaussians .......... : [|] 6476/6636[-8GFitting islands with Gaussians .......... : [] 6636/6636[-10G

stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device


Total number of Gaussians fit to image .. : 7037
Total flux density in model ............. : 0.043 Jy
stty: 'standard input': Inappropriate ioctl for device
--> Grouping Gaussians into sources
Number of sources formed from Gaussians   : 6718
    Island #3 (x=0, y=2667): fit with 1 Gaussian with flag = 278
    Island #4 (x=0, y=2713): fit with 1 Gaussian with flag = 4
    Island #107 (x=56, y=394): fit with 1 Gaussian with flag = 256
    Island #197 (x=110, y=2479): fit with 1 Gaussian with flag = 64
    Island #342 (x=192, y=150): fit with 1 Gaussian with flag = 74
    Island #371 (x=185, y=503): fit with 1 Gaussian with flag = 256
    Island #372 (x=186, y=838): fit with 1 Gaussian with flag = 256
    Island #408 (x=202, y=1908): fit with 1 Gaussian with flag = 268
    Island #419 (x=211, y=2552): fit with 1 Gaussian with flag = 256
    Island #425 (x=215, y=699): fit with 1 Gaussian with flag = 256
    Island #518 (x=259, y=2959): fit with 1 Gaussian with flag = 256
    Island #555 (

In [8]:
# 3) Train classifiers for each frequency's source DataFrame:
print("\nStep 3: Training classifiers")
classifiers = {}
for freq, source_train_df in sources_training.items():
    print(freq)
    # Load truth catalogue for the training area into memory
    print(source_train_df.head(3))
    train_truth_cat_df = load_truth_df(train_truth_path(freq), skiprows=18)

    # Construct and train classifier
    classifier = SKLClassification(
        algorithm=RandomForestClassifier,
        classifier_kwargs={"n_estimators": 100, "class_weight": "balanced"},
    )
    srl_df = classifier.train(
        source_train_df, train_truth_cat_df, regressand_col="class_t", freq=freq
    )
    print(srl_df.shape)
    # Store model for prediction later
    classifiers[freq] = classifier

    # (Optional) Write model to disk; allows later loading without retraining.
    model_dir = os.path.dirname(model_path(freq))
    Path(model_dir).mkdir(parents=True, exist_ok=True)
    classifier.save_model(model_path(freq))




Step 3: Training classifiers
1400
   Source_id  Isl_id          RA      E_RA        DEC     E_DEC    Total_flux  \
0          0       0  359.999746  0.000014 -29.856456  0.000011  9.947102e-07   
1          1       1  359.999413  0.000017 -29.828395  0.000009  4.765558e-06   
2          2       2  359.999746  0.000005 -29.794905  0.000009  2.980899e-06   

   E_Total_flux     Peak_flux   E_Peak_flux  ...  DC_PA_img_plane  \
0  1.908813e-07  6.020551e-07  7.738053e-08  ...        64.362563   
1  4.187382e-07  9.121587e-07  6.799415e-08  ...       104.110879   
2  2.660662e-07  1.288173e-06  8.365145e-08  ...        10.713707   

   E_DC_PA_img_plane  Isl_Total_flux  E_Isl_Total_flux       Isl_rms  \
0          25.682843    5.253248e-07      8.197129e-08  7.220221e-08   
1           8.138976    3.112229e-06      1.788294e-07  7.380012e-08   
2           8.554466    1.931456e-06      1.303469e-07  7.901944e-08   

       Isl_mean  Resid_Isl_rms  Resid_Isl_mean  S_Code  n_gaussians  
0 -3

In [ ]:
for freq, source_train_df in sources_training.items():
    print(source_train_df.head(2))
    print(source_train_df.shape)    

   Source_id  Isl_id          RA      E_RA        DEC     E_DEC    Total_flux  \
0          0       0  359.999746  0.000014 -29.856456  0.000011  9.947102e-07   
1          1       1  359.999413  0.000017 -29.828395  0.000009  4.765558e-06   

   E_Total_flux     Peak_flux   E_Peak_flux  ...  DC_PA_img_plane  \
0  1.908813e-07  6.020551e-07  7.738053e-08  ...        64.362563   
1  4.187382e-07  9.121587e-07  6.799415e-08  ...       104.110879   

   E_DC_PA_img_plane  Isl_Total_flux  E_Isl_Total_flux       Isl_rms  \
0          25.682843    5.253248e-07      8.197129e-08  7.220221e-08   
1           8.138976    3.112229e-06      1.788294e-07  7.380012e-08   

       Isl_mean  Resid_Isl_rms  Resid_Isl_mean  S_Code  n_gaussians  
0 -3.125654e-08   1.380381e-08   -3.114713e-08       S            1  
1 -2.670467e-08   9.398234e-08   -4.783774e-08       S            1  

[2 rows x 46 columns]
(6718, 46)


In [8]:
# this one takes long to run
# 4) Source finding (full):
sources_full = {}
print("\nStep 4: Source finding (full)")
for sdc1_image in sdc1_image_list:
    source_finder = SourceFinder(sdc1_image.pb_corr_image)
    sources_full[sdc1_image.freq] = source_finder.run()

    # Remove temp files:
    source_finder.reset()

#this step takes quite some time

--> Opened '1400mhz_1000h_pbcor.fits'
Image size .............................. : (5204, 4776) pixels
Number of channels ...................... : 1
Number of Stokes parameters ............. : 1



Step 4: Source finding (full)


Beam shape (major, minor, pos angle) .... : (1.66667e-04, 1.66667e-04, 0.0) degrees
Frequency of image ...................... : 1400.000 MHz
Number of blank pixels .................. : 0 (0.0%)
Flux from sum of (non-blank) pixels ..... : 0.037 Jy
--> Calculating background rms and mean images
/idia/projects/hack4dev/dc1/vDC1/lib/python3.8/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)
Using user-specified rms_box ............ : (74, 19) pixels
--> Using 2D map for background rms
--> Variation in mean image significant
--> Using 2D map for background mean
Min/max values of background rms map .... : (6.23e-08, 1.37e-05) Jy/beam
Min/max values of background mean map ... : (-6

Fitting islands with Gaussians .......... : [|] 0/14812

stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device


...

stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input'stty: 'standard input': Inappropriate ioctl for device
: Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: stty: 'standard input''standard input': Inappropriate ioctl for device
: Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'sta

Fitting islands with Gaussians .......... : [/] 13774/14812Fitting islands with Gaussians .......... : [/] 13774/14812[-7G[-7G[-7GFitting islands with Gaussians .......... : [-] 13775/14812[-7G--Fitting islands with Gaussians .......... : [|] 13777/14812[-7G\-Fitting islands with Gaussians .......... : [|] 13777/14812Fitting islands with Gaussians .......... : [|] 13777/14812[-7G/[-7G--Fitting islands with Gaussians .......... : [-] 13779/14812Fitting islands with Gaussians .......... : [-] 13779/14812[-7G[-7G|Fitting islands with Gaussians .......... : [\] 13780/14812Fitting islands with Gaussians .......... : [-] 13780/14812[-7G[-7GFitting islands with Gaussians .......... : [/] 13782/14812Fitting islands with Gaussians .......... : [-] 13783/14812-[-7G-[-7GFitting islands with Gaussians .......... : [-] 13783/14812[-7G|//Fitting islands with Gaussians .......... : [|] 13785/14812[-7G/Fitting islands with Gaussians .......... : [-] 13787/14812-[-7GFitting islands with

stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'sta

/Fitting islands with Gaussians .......... : [|] 13805/14812[-8G-Fitting islands with Gaussians .......... : [|] 13809/14812

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
# %store sources_full

Stored 'sources_full' (dict)


In [3]:
%store -r sources_full

In [10]:
# 5) Source classification (full)
print("\nStep 5: Classification")
for freq, source_df in sources_full.items():
    print(source_df)
    source_df["class"] = classifiers[freq].test(source_df)
    class_prob = classifiers[freq].predict_proba(source_df)
    source_df["class_prob"] = np.amax(class_prob, axis=1)

    # (Optional) Write source list DataFrame to disk
    write_df_to_disk(source_df, full_source_df_path(freq))




Step 5: Classification
       Source_id  Isl_id          RA      E_RA        DEC     E_DEC  \
0              0       0    0.066975  0.000004 -29.932994  0.000005   
1              1       1    0.066921  0.000002 -29.922052  0.000001   
2              2       3    0.065637  0.000001 -29.829974  0.000001   
3              3       3    0.064335  0.000008 -29.829784  0.000006   
4              4       3    0.064787  0.000004 -29.829553  0.000003   
...          ...     ...         ...       ...        ...       ...   
15008      15008   14806  359.664393  0.000007 -29.915872  0.000017   
15009      15009   14807  359.664226  0.000001 -29.912475  0.000002   
15010      15010   14808  359.664025  0.000011 -29.944404  0.000010   
15011      15011   14809  359.664421  0.000018 -29.813855  0.000014   
15012      15012   14810  359.663792  0.000010 -29.983458  0.000012   

       Total_flux  E_Total_flux     Peak_flux   E_Peak_flux  ...  \
0        0.000002  1.572058e-07  1.357248e-06  7.493803

In [10]:
# 6) Create final catalogues and calculate scores
print("\nStep 6: Final score")
for freq, source_df in sources_full.items():
    # Assemble submission and truth catalogues for scoring
    sub_cat_df = cat_df_from_srl_df(source_df, guess_class=False)
    truth_cat_df = load_truth_df(full_truth_path(freq), skiprows=0)
    print(sub_cat_df)
    print(truth_cat_df)
    
#     # (Optional) Write final submission catalogue to disk
#     write_df_to_disk(sub_cat_df, submission_df_path(freq))

#     # Calculate score
#     scorer = Sdc1Scorer(sub_cat_df, truth_cat_df, freq)
#     score = scorer.run(mode=0, train=False, detail=True)

#     # Write short score report:
#     score_path = score_report_path(freq)
#     score_dir = os.path.dirname(score_path)
#     Path(score_dir).mkdir(parents=True, exist_ok=True)

#     with open(score_path, "w+") as report:
#         report.write(
#             "Image: {}, frequency: {} MHz\n".format(image_path(freq), freq)
#         )
#         report.write("Score was {}\n".format(score.value))
#         report.write("Number of detections {}\n".format(score.n_det))
#         report.write("Number of matches {}\n".format(score.n_match))
#         report.write(
#             "Number of matches too far from truth {}\n".format(score.n_bad)
#         )
#         report.write("Number of false detections {}\n".format(score.n_false))
#         report.write("Score for all matches {}\n".format(score.score_det))
#         report.write("Accuracy percentage {}\n".format(score.acc_pc))
#         report.write("Classification report: \n")
#         report.write(
#             classification_report(
#                 score.match_df["class_t"],
#                 score.match_df["class"],
#                 labels=[1, 2, 3],
#                 target_names=["1 (SS-AGN)", "2 (FS-AGN)", "3 (SFG)"],
#                 digits=4,
#             )
#         )

# print("\nComplete")


Step 6: Final score
          id   ra_core   dec_core   ra_cent   dec_cent      flux  \
0          0  0.066975 -29.932994  0.066975 -29.932994  0.000002   
1          1  0.066921 -29.922052  0.066921 -29.922052  0.000011   
2          2  0.065675 -29.830004  0.065637 -29.829974  0.000101   
3          3  0.064335 -29.829784  0.064335 -29.829784  0.000014   
4          4  0.064787 -29.829553  0.064787 -29.829553  0.000003   
...      ...       ...        ...       ...        ...       ...   
15008  15008 -0.335607 -29.915872 -0.335607 -29.915872  0.000001   
15009  15009 -0.335774 -29.912475 -0.335774 -29.912475  0.000008   
15010  15010 -0.335975 -29.944404 -0.335975 -29.944404  0.000002   
15011  15011 -0.335579 -29.813855 -0.335579 -29.813855  0.000003   
15012  15012 -0.336208 -29.983458 -0.336208 -29.983458  0.000001   

          core_frac     b_maj     b_min          pa  size  class  
0      4.317136e-07  0.748702  0.633776   33.987021     2    3.0  
1      5.610104e-06  1.09802

In [9]:
print(1)

1


In [10]:
%store source_train_df
%store source_df



Stored 'source_train_df' (DataFrame)
Stored 'source_df' (DataFrame)


In [11]:
%store srl_df

Stored 'srl_df' (DataFrame)


In [ ]:
source_df